## Validation Of RV's
---
`corv` radial velocities were validated on the high-resolution spectra from Falcon, et.al **talk about validation and all that jazz once you get it done.**

In [2]:
# Install a pip package in the current Jupyter kernel
import sys
import os
sys.path.append('../../')
sys.path.append('../../wdtools')
sys.path.append('../../WD_models')
sys.path.append('../../corv/src')


### General
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import astropy
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u
from astropy.table import Table, Column, MaskedColumn, join

from tqdm import tqdm
import pandas as pd
from scipy.optimize import curve_fit
import scipy.stats as stats

### Tools
import wdtools
import WD_models
import data_selector as ds
import WD_models
from bisect import bisect_left
import lmfit
import glob

from astroquery.sdss import SDSS
from astroquery.gaia import Gaia

import corv

#corv.sdss.make_catalogs()

using local paths
star and exposure catalogs not found! check paths and run make_catalogs() if you want to use sdss functionality. otherwise ignore.


In [6]:
c = 2.998e10
k = 1.38e-16
h = 6.626e-27
halpha = 6564.61
hbeta = 4862.68
hgamma = 4341.68
hdelta = 4102.89
speed_light = 299792458 #m/s
radius_sun = 6.957e8
mass_sun = 1.9884e30
newton_G = 6.674e-11
pc_to_m = 3.086775e16

In [4]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, join_skycoord
from astropy import table

catalog = Table.read('../data/vedanta/wd_catalog.fits')
falcon = Table.read('../data/falcon2010.fit')

falcon['loc'] = SkyCoord(ra=falcon['_RA'], dec=falcon['_DE'])
catalog['loc'] = SkyCoord(ra=catalog['ra']*u.deg, dec=catalog['dec']*u.deg)

join_func = table.join_skycoord(5 * u.arcsecond)
catalog = table.join(catalog, falcon, join_funcs={'loc': join_skycoord(5 * u.arcsec)})

from astropy.table import unique, Table

catalog = unique(catalog, keys = ['loc_id'])

In [5]:
catalog['corv_rv'] = -9999 * np.ones(len(catalog))
catalog['corv_erv'] = -9999 * np.ones(len(catalog))

In [8]:
print(catalog.keys())

['loc_id', 'dist', 'solution_id', 'designation', 'source_id', 'random_index', 'ref_epoch', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_weight_al', 'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error', 'mean_varpi_factor_al', 'astrometric_matched_observations', 'visibility_periods_used', 'astrometric_sigma5d_max', 'frame_rotator_object_type', 'matched_observations', 'duplicated_source', 'phot_g_n_obs', 'phot_g_mean_flux', 'pho

In [ ]:
for j in tqdm( range(len(catalog[:200]))):
    p,m,f = np.array(catalog['col_p_m_f'][j].split('-')).astype(float)
    
    try:
        xid = SDSS.query_specobj(plate = p, mjd = m, fiberID = f)
    except ValueError:
        print('unknown error')
        
    if xid == None:
        print('object not found in SDSS... skipping...')
        continue
    try:
        sp = SDSS.get_spectra(matches=xid)
    except:
        print('http error')
        continue
    for i in range(len(sp[0:1])):
        wl = np.array(10**sp[i][1].data['loglam'])
        fl = np.array(sp[i][1].data['flux'])
        ivar = np.array(sp[i][1].data['ivar'])
        
        corvmodel = corv.models.make_koester_model(names = ['a','b','g','d'])
        param_res, rv_res, rv_init = corv.fit.fit_corv(wl, fl, ivar, corvmodel, teff = catalog['teff'][j])
        print(catalog['vr'][j])
        
    catalog['corv_rv'][j] = (rv_res.params['RV'].value)
    catalog['corv_erv'][j] = (rv_res.params['RV'].stderr)

  0%|                                                                                    | 0/53 [00:00<?, ?it/s]

starting...


  2%|█▍                                                                       | 1/53 [03:01<2:36:58, 181.13s/it]

The chain is shorter than 50 times the integrated autocorrelation time for 1 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [28.50685843]
final params:
[[Fit Statistics]]
    # fitting method   = emcee
    # function evals   = 50000
    # data points      = 629
    # variables        = 1
    chi-square         = 635.6865
    reduced chi-square = 1.01223969
    Akaike info crit   = 8.65123310
    Bayesian info crit = 13.0953644
[[Variables]]
    teff:  15028 (fixed)
    logg:  8.141378 (fixed)
    RV:    49.2505762 +/- 4.41809810 (8.97%) (init = 35.76215)
    res:   1 (fixed)
rv init: 35.76214965099458
52.0
starting...


  4%|██▊                                                                      | 2/53 [06:18<2:41:50, 190.40s/it]

...

The chain is shorter than 50 times the integrated autocorrelation time for 1 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [21.78582881]
final params:
[[Fit Statistics]]
    # fitting method   = emcee
    # function evals   = 50000
    # data points      = 629
    # variables        = 1
    chi-square         = 517.3214
    reduced chi-square = 0.82376021
    Akaike info crit   = -120.948674
    Bayesian info crit = -116.504543
[[Variables]]
    teff:  16823 (fixed)
    logg:  8.667584 (fixed)
    RV:    56.9219986 +/- 4.87996951 (8.57%) (init = 56.30615)
    res:   1 (fixed)
rv init: 56.30614899272348
69.0
starting...


  9%|██████▉                                                                  | 5/53 [15:36<2:28:28, 185.60s/it]

The chain is shorter than 50 times the integrated autocorrelation time for 1 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [33.85903294]
final params:
[[Fit Statistics]]
    # fitting method   = emcee
    # function evals   = 50000
    # data points      = 629
    # variables        = 1
    chi-square         = 354.16278
    reduced chi-square = 0.56395347
    Akaike info crit   = -359.281634
    Bayesian info crit = -354.837503
[[Variables]]
    teff:  33443 (fixed)
    logg:  7.751415 (fixed)
    RV:    57.7253671 +/- 5.39966155 (9.35%) (init = 81.16269)
    res:   1 (fixed)
rv init: 81.16268575089379
58.0
starting...
The chain is shorter than 50 times the integrated autocorrelation time for 1 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [20.1576517]
final params:
[[Fit Statistics]]
    # fitting method   = emcee
    # function evals   = 50000
    # data points      = 629
    # variables        = 1
    chi

 11%|████████▎                                                                | 6/53 [18:49<2:27:15, 187.99s/it]

starting...
